In [57]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,
KNeighborsClassifier)

In [58]:
postmates = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/postmates_all.csv')
postmates = postmates.loc[:, ~postmates.columns.str.contains('^Unnamed')]

In [59]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.40,"(40.7207158, -73.9947851)",40.720716,-73.994785
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.30,"(40.7207158, -73.9947851)",40.720716,-73.994785
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Chai Latte,"Chai extract with your choice of milk, poured ...",NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Athletes Burrito (D),"Egg whites, sauteed spinach, roasted tomatoes,...",15.40,"(40.7207158, -73.9947851)",40.720716,-73.994785
...,...,...,...,...,...,...,...,...,...,...
41378,Hudson Food Market,DELI,0.0,312 11TH AVE,Tic Tac,NaN,1.99,"(40.7527815, -74.00323078347762)",40.752781,-74.003231
41379,Hudson Food Market,DELI,0.0,312 11TH AVE,Starburst,NaN,1.69,"(40.7527815, -74.00323078347762)",40.752781,-74.003231
41380,Hudson Food Market,DELI,0.0,312 11TH AVE,Chunky Candy,NaN,1.69,"(40.7527815, -74.00323078347762)",40.752781,-74.003231
41381,Hudson Food Market,DELI,0.0,312 11TH AVE,Sour Power Straws Candy,NaN,1.69,"(40.7527815, -74.00323078347762)",40.752781,-74.003231


In [60]:
postmatesSub = postmates[['Name','MenuItemPrice','Category','MenuItem','MenuItemDescription']]

# Split Menu Item Description By Word

In [61]:
postmatesSub['MenuItemDescription'] = list(map(lambda row: str(row).replace(',','').split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItemDescription']))




<ipython-input-61-7d6af560d296>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postmatesSub['MenuItemDescription'] = list(map(lambda row: str(row).replace(',','').split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItemDescription']))


In [62]:
postmatesSub['MenuItem'] = list(map(lambda row: str(row).split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItem']))


<ipython-input-62-7f7ee7678721>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postmatesSub['MenuItem'] = list(map(lambda row: str(row).split(' ') if pd.notnull(row) else [row], postmatesSub['MenuItem']))


In [63]:
postmatesSub['Category'] = list(map(lambda row: [row] if pd.notnull(row) else [row], postmatesSub['Category']))

<ipython-input-63-48c0dd3d55ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postmatesSub['Category'] = list(map(lambda row: [row] if pd.notnull(row) else [row], postmatesSub['Category']))


In [64]:
postmates['BagOfWords'] = postmatesSub['Category'] + postmatesSub['MenuItem'] + postmatesSub['MenuItemDescription']



In [65]:
postmates['BagOfWords'] = list(map(lambda row: [str(each_word).lower() for each_word in row], postmates['BagOfWords']))


In [66]:
postmates['BagOfWords'] = list(map(lambda row: ' '.join(row), postmates['BagOfWords']))

In [67]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude,BagOfWords
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.40,"(40.7207158, -73.9947851)",40.720716,-73.994785,breakfast bl burrito (d) scrambled eggs bacon ...
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.30,"(40.7207158, -73.9947851)",40.720716,-73.994785,breakfast avocado smash (d) our classic avo sm...
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,breakfast iced latte double shot of our signat...
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Chai Latte,"Chai extract with your choice of milk, poured ...",NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785,breakfast iced chai latte chai extract with yo...
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Athletes Burrito (D),"Egg whites, sauteed spinach, roasted tomatoes,...",15.40,"(40.7207158, -73.9947851)",40.720716,-73.994785,breakfast athletes burrito (d) egg whites saut...
...,...,...,...,...,...,...,...,...,...,...,...
41378,Hudson Food Market,DELI,0.0,312 11TH AVE,Tic Tac,NaN,1.99,"(40.7527815, -74.00323078347762)",40.752781,-74.003231,deli tic tac nan
41379,Hudson Food Market,DELI,0.0,312 11TH AVE,Starburst,NaN,1.69,"(40.7527815, -74.00323078347762)",40.752781,-74.003231,deli starburst nan
41380,Hudson Food Market,DELI,0.0,312 11TH AVE,Chunky Candy,NaN,1.69,"(40.7527815, -74.00323078347762)",40.752781,-74.003231,deli chunky candy nan
41381,Hudson Food Market,DELI,0.0,312 11TH AVE,Sour Power Straws Candy,NaN,1.69,"(40.7527815, -74.00323078347762)",40.752781,-74.003231,deli sour power straws candy nan


In [68]:
count = CountVectorizer()
countVec = count.fit(postmates['BagOfWords'])
count_matrix = countVec.transform(postmates['BagOfWords'])

In [156]:
def getMostSimilarItems(user_input):
    user_input = [user_input]
    count_matrix_inp = countVec.transform(user_input)
    
    cosine_sim = cosine_similarity(count_matrix, count_matrix_inp)
    most_similar = cosine_sim[:,0].argsort(axis=0)[::-1]

    similar = postmates.iloc[most_similar][['Name','Category','MenuItem','Favorites']]
    
    return pd.DataFrame(similar)


In [157]:
most_similar = getMostSimilarItems('Chicken parm')
most_similar.head(25)

,Name,Category,MenuItem,Favorites
12061,Parm,ITALIAN,Chicken Parm Hero,535.0
12074,Parm,ITALIAN,Chicken Parm Platter,535.0
12071,Parm,ITALIAN,Chicken Parm Roll,535.0
38590,Smiley's,DELI,4. Chicken Parm Panini,0.0
26287,Midnight Express,BURGER,Chicken Parm Hero Sandwich,312.0
32582,Healthy Market Food and Deli,DELI,Chicken Parm Panini,0.0
22505,B & W Deli & Pizzeria,PIZZA,Chicken Parm Pizza,0.0
12160,Cutlets Sandwich Co.,SANDWICH,Number Four,0.0
32520,Healthy Market Food and Deli,DELI,Chicken Parm Sandwich,0.0
22811,Famous Original Ray's Pizza,PIZZA,Chicken & Tomato Pizza Pie,143.0
